## Climatic data



In [53]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# data_folder = 'projects/amazon-forest-regrowth/assets'
roi = (
    ee.FeatureCollection("projects/amazon-forest-regrowth/assets/raw/biomes_br")
    .geometry()
    .dissolve()
)

years = range(1985, 2020)

ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)

# Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality

In [54]:
terraclim = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterDate("1985-01-01", "2019-12-31")
    .map(lambda image: image.clip(roi))
    .map(
        lambda image: image.reduceResolution(
            ee.Reducer.median(), bestEffort=True, maxPixels=1024
        ).reproject(crs="EPSG:4326", scale=10000)
    )
)

# scaling to 0.1
maxtemp = terraclim.select("tmmx").map(lambda image: image.multiply(0.1))
mintemp = terraclim.select("tmmn").map(lambda image: image.multiply(0.1))
radiation = terraclim.select("srad").map(lambda image: image.multiply(0.1))
prec = terraclim.select("pr")


def export_image(img, name):
    # Create the export task
    task = ee.batch.Export.image.toAsset(
        image=img.slice(1),
        description=f"{name}",
        assetId=f"projects/amazon-forest-regrowth/assets/{name}",
        scale=10000,
        crs="EPSG:4326",
        maxPixels=4e12,
    )
    # Start the export task
    task.start()

## Precipitation

For each regrowing forest pixel, we only consider rainfall AFTER the regrowth moment.
- Make a raster with mean yearly precipitation, temperature, and precipitation
- select only the values for years after regrowth. For years before regrowth, change the values to zero.

## Seasonality index

Walsh and Lawler 1981

In [55]:
yearly_SI = ee.Image()
yearly_mean_prec = ee.Image()
yearly_mean_temp = ee.Image()

for year in years:  # get mean monthly precipitation for the year
    yr_prec = prec.filter(ee.Filter.calendarRange(year, year, "year"))
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    yearly_mean_prec = yearly_mean_prec.addBands(
        mean_prec.toFloat().rename(f"prec_{year}")
    )
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month: month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    yearly_SI = yearly_SI.addBands(
        sum_deviations.divide(total_annual_prec).toFloat().rename(f"si_{year}")
    )
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(year, year, "year")).reduce(
        ee.Reducer.mean()
    )
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(year, year, "year")).reduce(
        ee.Reducer.mean()
    )
    # Calculate the mean temperature directly
    yearly_mean_temp = yearly_mean_temp.addBands(
        yearly_maxtemp.add(yearly_mintemp)
        .divide(2)
        .toFloat()
        .rename(f"mean_temp_{year}")
    )

export_image(yearly_mean_prec, "yearly_mean_prec")
export_image(yearly_SI, "yearly_SI")
export_image(yearly_mean_temp, "yearly_mean_temp")

## Yearly CWD

Calculated as in Malhi et al 2009 and Aragão et al 2007, considering ET fixed as 100mm/month.

[equations](https://imgur.com/o4lVmM7)

In [52]:
# """
# Calculate monthly cumulative water deficit
# """
# prec_list = prec.toList(prec.size())
# months_list = range(2, prec.size().getInfo()+1)
# cwd_monthly = prec_list.get(0)

# for month in months_list:
#     current_image = prec_list.get(month)
#     previous_val = cwd_monthly.select(-1)
#     result_image = previous_val.add(current_image).subtract(100)
#     cwd_monthly = cwd_monthly.addBands(result_image.where(result_image.gt(0), 0).toFloat())

#                                    #.set('system:time_start', current_image.get('system:time_start'))
# cwd_yearly = ee.Image()

for year in years:
    print(year)
    # Calculate the start and end indices for the current year
    start = (year - 1985) * 12
    end = start + 12
    print(start)
    print(end)
    # # Slice the list to get the images for the current year
    # yr = ee.List(cwd_list).slice(start, end)
    # cwd_a = ee.ImageCollection.fromImages(yr)
    # mcwd_a = cwd_a.reduce(ee.Reducer.min())
    # return mcwd_a.set('year', year)

# # Map the function over the range
# yearly_mcwd_img = ee.ImageCollection.fromImages(years.map(mcwd_yearly)).toBands()

# yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
# new_band_names = ['mcwd_{}'.format(year) for year in yearlist] # Append 'si_' to each year
# yearly_mcwd_img = yearly_mcwd_img.rename(new_band_names)

1985
0
12
1986
12
24
1987
24
36
1988
36
48
1989
48
60
1990
60
72
1991
72
84
1992
84
96
1993
96
108
1994
108
120
1995
120
132
1996
132
144
1997
144
156
1998
156
168
1999
168
180
2000
180
192
2001
192
204
2002
204
216
2003
216
228
2004
228
240
2005
240
252
2006
252
264
2007
264
276
2008
276
288
2009
288
300
2010
300
312
2011
312
324
2012
324
336
2013
336
348
2014
348
360
2015
360
372
2016
372
384
2017
384
396
2018
396
408
2019
408
420
